# llama3-scheduling

## Design of Experiments for Model Evaluation

This notebook is designed to perform a systematic evaluation of different configurations of Llama 3 models using the principles of **Design of Experiments (DOE)**. The goal is to explore how various hyperparameter settings affect the performance of text generation tasks, ensuring a comprehensive analysis of each configuration.

## Objectives
1. **Parameter Exploration**: Evaluate multiple configurations by varying:
   - Model sizes (e.g., 1B, 3B, 8B).
   - Batch sizes for data processing.
   - Maximum tokens generated per response.
   - Quantization precision levels (e.g., 4-bit, 8-bit).
2. **Performance Metrics**: Measure the quality of the generated text using metrics like ROUGE.
3. **Result Logging**: Store all results systematically in a CSV file for further analysis.

## Workflow
1. **Environment Setup**: Install necessary dependencies and configure CUDA for efficient computation.
2. **Dataset Loading and Preprocessing**: Prepare input text data and format it for the rewriting task.
3. **Experimental Design**:
   - Define a full-factorial experiment setup to evaluate all possible parameter combinations.
   - Use a combination of models, batch sizes, token limits, and quantization settings to generate configurations.
4. **Model Evaluation**:
   - Load the specified model and tokenizer for each configuration.
   - Tokenize input text and process it through the model.
   - Generate outputs and compute evaluation metrics for the text.
5. **Result Logging**: Save results, including performance metrics and timing information, into a CSV file.

## Outputs
- A CSV file (`output.csv`) containing detailed information about each experiment, including:
  - Model configuration.
  - Generated outputs and their corresponding inputs.
  - ROUGE scores for evaluating text quality.
  - Execution times for each configuration.

### Environment Setup
This section installs all the necessary dependencies required for the notebook. These libraries include tools for natural language processing, dataset management, and statistical analysis. If you already have a compatible environment, you can skip this step.

In [ ]:
# Install dependencies (same as the env file, so you may wish to skip this if running locally / with persistent conda environment)
%pip install transformers
%pip install accelerate
%pip install bitsandbytes
%pip install pyDOE3
%pip install datasets
%pip install torch
%pip install jinja2
%pip install torchmetrics
%pip install tqdm
%pip install jupyter
%pip install pretty-jupyter
%pip install researchpy
%pip install statsmodels
%pip install matplotlib
%pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

### Library Imports
Here, we import the essential Python libraries and modules that will be used throughout the notebook. These include:
- `transformers` and `datasets` for natural language processing tasks.
- `pyDOE3` for design of experiments.
- `torchmetrics` for evaluation metrics.
- `jinja2` for templating.
- `statsmodels` for statistical modeling.

In [ ]:
import pandas as pd
import csv
import tqdm
import time
import os
import jinja2
import textwrap
import datasets

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from torch.utils.data import DataLoader
from pyDOE3 import *
from torchmetrics.text.rouge import ROUGEScore

import statsmodels.api as sm
from statsmodels.formula.api import ols


### Data Loading
This cell loads the dataset, `openrewriteeval_aaai24.csv`, into a Pandas DataFrame for further processing. The dataset is expected to contain text samples that will be rewritten using a template.

In [ ]:
df = pd.read_csv("drive/MyDrive/openrewriteeval_aaai24.csv")
df

,source,target,comment,task
0,Caught red handed by the pedestrian’s roadbloc...,I was impeded by the sight of a red hand of th...,write a story about a person who is always late,others
1,If there were any suggestions of Mendelssohn i...,If there were any hints of Mendelssohn lurking...,write in style of roger ebert,others
2,Deep Tissue Massage is a form of bodywork that...,Deep Tissue Massage soothes tension in the bod...,add a call to action,others
3,The procedure for constructing your resume and...,The procedures of building our resume through ...,make it more friendly and personal,others
4,A. Action of the Board: The Board upon a findi...,A. Action of the Board:\n-If the Board determi...,Use bullet points to make this more readable,others
...,...,...,...,...
1624,Singles athletes are expected to submit their ...,Our club expects singles athletes to deliver t...,Paraphrase,paraphrase
1625,Test design is important. Performance metrics ...,Test design is vital. Performance metrics shou...,Rephrase this text,paraphrase
1626,The control of fire was the first great discov...,The ability to manage fire was the first great...,Paraphrase this,paraphrase
1627,There are various types of identity which show...,Different kinds of identity reveal how a perso...,Reword this text,paraphrase


### Template Creation and Data Processing
We define a Jinja2 template for generating text instructions for the rewriting task. The dataset is preprocessed to include the formatted instructions, which will later be used for model evaluation.

In [ ]:
example_template = jinja2.Template(
    textwrap.dedent(
        """Instruction: Rewrite the given text while keeping the same meaning. Do not write anything else!

        Given text: {{ review }}

        Rewritten text: """
    )
)

df["processed"] = df["source"].apply(lambda x: example_template.render(review=x))

ds = datasets.Dataset.from_pandas(df[:10])
ds

Dataset({
    features: ['source', 'target', 'comment', 'task', 'processed'],
    num_rows: 10
})

### Experimental Design
In this section, we define the parameters for our experiments:
- **Models:** Variants of Llama 3.x models with different parameter sizes.
- **Batch Sizes:** The number of samples processed simultaneously.
- **Token Limits:** Maximum number of tokens generated in each output.
- **Precision Settings:** Different levels of numerical precision (e.g., 4-bit and 8-bit).

Using a full factorial design, we generate all possible combinations of these parameters for systematic evaluation.

In [ ]:
models = [
  "meta-llama/Llama-3.2-1B",
  "meta-llama/Llama-3.2-3B",
  "meta-llama/Llama-3.1-8B",
]

batch_sizes = [
  1,
  4,
  8,
]

max_new_tokens_list = [
    100,
    250,
    500,
]

load_in_bits = [4, 8, None]

configurations = fullfact([
    len(models),
    len(batch_sizes),
    len(max_new_tokens_list),
    len(load_in_bits)
])

configurations

array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [2., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 1., 0., 0.],
       [2., 1., 0., 0.],
       [0., 2., 0., 0.],
       [1., 2., 0., 0.],
       [2., 2., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [2., 0., 1., 0.],
       [0., 1., 1., 0.],
       [1., 1., 1., 0.],
       [2., 1., 1., 0.],
       [0., 2., 1., 0.],
       [1., 2., 1., 0.],
       [2., 2., 1., 0.],
       [0., 0., 2., 0.],
       [1., 0., 2., 0.],
       [2., 0., 2., 0.],
       [0., 1., 2., 0.],
       [1., 1., 2., 0.],
       [2., 1., 2., 0.],
       [0., 2., 2., 0.],
       [1., 2., 2., 0.],
       [2., 2., 2., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.],
       [2., 0., 0., 1.],
       [0., 1., 0., 1.],
       [1., 1., 0., 1.],
       [2., 1., 0., 1.],
       [0., 2., 0., 1.],
       [1., 2., 0., 1.],
       [2., 2., 0., 1.],
       [0., 0., 1., 1.],
       [1., 0., 1., 1.],
       [2., 0., 1., 1.],
       [0., 1., 1., 1.],


In [ ]:
# Enable dynamic CUDA shared allocations and enforce error reporting for CUDA operations
os.environ["TORCH_USE_CUDA_DSA"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Initialize ROUGE metric scorer for text evaluation
rouge = ROUGEScore()

# Initialize a list to store the results of all evaluations
outputs = []

# Open a CSV file to log the results of the experiments
with open("drive/MyDrive/output.csv", 'w', newline='') as csvfile:
  # Define the column names for the output CSV
  fieldnames = [
      'model',
      'batch_size',
      'max_new_tokens',
      'input',
      "raw",
      'output',
      'trimmed_output',
      'score',
      'start_time',
      'end_time',
      'time_taken',
      "quantization"
  ]
  # Create a CSV writer with a custom delimiter and quoting style
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='§', quoting=csv.QUOTE_MINIMAL)
  # Write the header row to the CSV file
  writer.writeheader()

  # Iterate over all generated configurations for the experiment
  for configuration in tqdm.tqdm(configurations):
    # Retrieve parameters based on the current configuration
    model_name = models[int(configuration[0])]
    batch_size = batch_sizes[int(configuration[1])]
    max_new_tokens = max_new_tokens_list[int(configuration[2])]
    quantization_bits = load_in_bits[int(configuration[3])]

    # Log the current configuration
    print(f"Model: {model_name}")
    print(f"Batch size: {batch_size}")
    print(f"Max new tokens: {max_new_tokens}")
    print(f"Quantization: {quantization_bits}")

    # Set up quantization configuration for the model
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=quantization_bits == 4,
        load_in_8bit=quantization_bits == 8
    )

    # Load the tokenizer with quantization and device mapping configurations
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype="auto",
        padding_side="left",
        quantization_config=quantization_config,
    )

    # Load the model with the same quantization and device configurations
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype="auto",
        quantization_config=quantization_config,
    )

    # Ensure the tokenizer has a pad token, fallback to EOS if missing
    tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

    # Create a DataLoader to batch the dataset for the current configuration
    dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False)

    # Iterate through each batch of the dataset
    for batch in tqdm.tqdm(dataloader):
      start_time = time.time() # Start timing the batch processing

      print("input", batch["processed"])

      # Tokenize the batch input text
      tokenized_text = tokenizer.batch_encode_plus(
          batch["processed"],
          return_tensors="pt",
          padding=True
      )

      # Prepare input IDs and attention masks on the appropriate device
      input_ids = tokenized_text["input_ids"].to(model.device)
      attention_mask = tokenized_text["attention_mask"].to(model.device)

      # Generate model outputs for the input text
      output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,      # Limit for text length
        temperature=1.0,                    # Lower for precise rewrites
        top_p=0.9,                          # Slightly tighter for coherence
        top_k=50                            # Focus on most probable tokens
    )

      # Decode the generated output text
      output_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
      end_time = time.time() # End timing

      # Compute the ROUGE score for the generated text
      score = rouge(output_text, batch["processed"])

      # Prepare the output record for the current batch
      output = {
          "model": model_name,
          "batch_size": batch_size,
          "max_new_tokens": max_new_tokens,
          "raw": batch["source"],
          "input": batch["processed"],
          "output": output_text,
          "trimmed_output": [output[len(input):] for input, output in zip(batch["processed"], output_text)],
          "score": score,
          "start_time": start_time,
          "end_time": end_time,
          "time_taken": end_time - start_time,
          "quantization": quantization_bits
      }

      # Append the result to the output list
      outputs.append(output)
      # Write the result to the CSV file
      writer.writerow(output)

      # Log the latest output for debugging purposes
      print(outputs[-1])
      print()

    print() # Add a blank line after processing each configuration

  0%|          | 0/81 [00:00<?, ?it/s]

Model: meta-llama/Llama-3.2-1B
Batch size: 1
Max new tokens: 100
Quantization: 4


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

...

{'model': 'meta-llama/Llama-3.1-8B', 'batch_size': 1, 'max_new_tokens': 500, 'raw': ['A. Action of the Board: The Board upon a finding by a majority vote that a member or chapter has violated the Articles of Incorporation or Bylaws shall notify the member or chapter of its delinquency in writing. If the delinquency is subject to being remedied, reasonable opportunity within the discretion of the Board for correction shall be allowed if however the breech is not subject to being corrected or is not corrected, the Board may by two-third (2/3) majority vote terminate or suspend for up to one (01) year. An individual member or chapter termination or Suspension by the Board does not become effective until that entire act has been affirmed by the four-fifths (4/5) vote of the Membership at the Annual or Special Meeting at which a quorum is present. In the event of a resolution calling for the termination of suspension of a member or chapter, the Board shall forthwith call a special meeting o


 60%|██████    | 6/10 [04:59<03:44, 56.12s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'model': 'meta-llama/Llama-3.1-8B', 'batch_size': 1, 'max_new_tokens': 500, 'raw': ['Juvo works by teaming up with mobile phone operators and financial institutions to enable members of underserved communities to build financial identities. The company’s technology uses credit algorithms based on advanced data science to find creditworthy individuals among what Polsky says are the “two billion underbanked people in the world – most of whom have a mobile phone” and to ensure their access to financial services. Juvo has been a beneficial partner, as well, helping its operators lower churn by 50% or more, increase average revenue per user (ARPU) by up to 15%, and realize an average increase in subscriber lifetime value of 65%.'], 'input': ['Instruction: Rewrite the given text while keeping the same meaning. Do not write anything else!\n\n        Given text: Juvo works by teaming up with mobile phone operators and financial institutions to enable members of underserved communities to buil


 70%|███████   | 7/10 [06:54<03:46, 75.45s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'model': 'meta-llama/Llama-3.1-8B', 'batch_size': 1, 'max_new_tokens': 500, 'raw': ['By all means, you are welcome to come to our office. Because Mr. Abrams is responsible for maintaining correspondence, via e-mail, with our many clients around the world as well as the thousands of individuals, such as yourself, who seek information over the Internet, Mr. Peter Krochak meets with potential clients in our office while Mr. Abrams looks after the e-mail. Should you wish to schedule an appointment, please telephone Mr. Krochak at +1-416-482-3387, Extension 23 (press 23 on your telephone when you hear the auto-attendant) between the hours of 09:00 and 17:00, Eastern Time, Monday to Friday. Generally speaking, Mr. Krochak charges a 300 CAD consultation fee for an appointment. If you subsequently become a client of Abrams & Krochak, the 300 CAD consultation fee will be deducted from Abrams & Krochak’s legal fees.'], 'input': ['Instruction: Rewrite the given text while keeping the same meanin